### Algorithm

### Continous optimization

In [229]:
import numpy as np
from numpy.random import randint
from numpy.random import rand

In [230]:
def objective_function(x):
    return x[0]**2 + x[1]**2

In [231]:
def decode(bounds, n_bits, bitstring):
    decoded = []
    largest = 2**n_bits
    for i in range(len(bounds)):
        start,end = i * n_bits, (i*n_bits)+n_bits
        substring = bitstring[start:end]
        chars = ''.join([str(s) for s in substring])
        integer = int(chars,2)
        
        value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
        decoded.append(value)
    return decoded

In [232]:
def selection(pop, scores, k=3):
	selection_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		if scores[ix] < scores[selection_ix]:
			selection_ix = ix
	return pop[selection_ix]

In [233]:
def crossover(p1,p2,r_cross):
	c1, c2 = p1.copy(), p2.copy()
	if rand() < r_cross:
		pt = randint(1, len(p1)-2)
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

In [234]:
def mutation(bitstring,r_mut):
    for i in range(len(bitstring)):
        if rand() < r_mut:
            bitstring[i] = 1 - bitstring[i]

In [235]:
def genetic_algorithm(objective_function,bounds,r_mut,n_iter=100,r_cross=0.9,n_bits=16,n_pop=100):
    
    pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
                         #n_bits
    best, best_eval = 0, objective_function(decode(bounds, n_bits, pop[0]))
                         #objective(pop[0])
    for gen in range(n_iter):

        decoded = [decode(bounds, n_bits, p) for p in pop]

        scores = [objective_function(d) for d in decoded]
		#scores = [objective(i) for i in pop]
         
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(f">iteration {gen}, new best f({decoded[i]}) = {scores[i]}")
	
        selected = [selection(pop, scores) for _ in range(n_pop)]
	
        children = list()
        for i in range(0, n_pop, 2):
			
            p1, p2 = selected[i], selected[i+1]
	
            for c in crossover(p1, p2, r_cross):
				
                mutation(c, r_mut)
			
                children.append(c)

        pop = children
    return [best, best_eval]

In [236]:
bounds = [[-5.0, 5.0], [-5.0, 5.0]]

r_mut = 1.0 / (float(n_bits) * len(bounds))

print(f'Starting genetic algorithm\n')
best, score = genetic_algorithm(objective_function, bounds, r_mut)
decoded = decode(bounds, n_bits, best)
print(f'\nGenetic algorithm completed\n')
print(f'Best solution: {decoded}')
print(f'Fitness score of the best solution: {score:.5f}')

Starting genetic algorithm

>iteration 0, new best f([-0.61859130859375, 0.08056640625]) = 0.3891461528837681
>iteration 0, new best f([-0.550994873046875, -0.019073486328125]) = 0.30395914800465107
>iteration 0, new best f([0.228118896484375, 0.174560546875]) = 0.08250961545854807
>iteration 1, new best f([0.071868896484375, 0.20538330078125]) = 0.047347438521683216
>iteration 2, new best f([0.071868896484375, 0.158538818359375]) = 0.03029969520866871
>iteration 3, new best f([0.071868896484375, 0.157470703125]) = 0.029962160624563694
>iteration 4, new best f([0.074310302734375, -0.0164794921875]) = 0.005793594755232334
>iteration 5, new best f([0.052337646484375, -0.0115966796875]) = 0.002873712219297886
>iteration 7, new best f([-0.028839111328125, 0.019378662109375]) = 0.001207226887345314
>iteration 8, new best f([0.01312255859375, -0.02655029296875]) = 0.0008771196007728577
>iteration 9, new best f([-0.028839111328125, 0.00518798828125]) = 0.000858609564602375
>iteration 9, new b

### Binary optimization

In [162]:
def onemax(x):
    return -sum(x)

In [186]:
def selection(pop,scores,k=3):
    selection_ix = np.random.randint(len(pop))
    for ix in range(0,len(pop),k-1):
        if scores[ix]<scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

In [196]:
def crossover(p1,p2,r_cross):
    c1,c2 = p1.copy(),p2.copy()
    if np.random.rand()<r_cross:
        ct = np.random.randint(len(p2)-1)
        c1 = p1[:ct]+p2[ct:]
        c2 = p1[ct:]+p2[:ct]
    return [c1,c2]

In [237]:
def mutation(bitstring,r_mut):
    for i in range(len(bitstring)):
        if np.random.rand()<r_mut:
            bitstring[i] = 1-bitstring[i]

In [238]:
def genetic_alg(onemax,bounds,r_cross,r_mut,n_iter=100,n_pop=100,n_bits=20):
    pop = [np.random.randint(0,2,n_bits).tolist() for _ in range(n_pop)]
    best,best_eval = 0,onemax(pop[0])
    for i in range(n_iter):
        score = [onemax(x) for x in pop]
        for j in range(n_pop):
            if score[j]<best_eval:
                best_eval,best = score[j],pop[j]
                print(f'{i}> f({best}) : {best_eval}')
            
        selected = [selection(pop,score) for _ in range(n_pop)]
        child = []
        for j in range(0,n_pop,2):
            p1,p2 = selected[j],selected[j+1]
            for c in crossover(p1,p2,r_cross):
                mutation(c,r_mut)
                child.append(c)
        pop = child
    return best,best_eval

In [239]:
bounds = [[-5.0, 5.0], [-5.0, 5.0]]

n_iter = 100

n_bits = 16

n_pop = 100

r_cross = 0.9

r_mut = 1.0 / (float(n_bits) * len(bounds))

print(f'Starting genetic algorithm\n')
best, score = genetic_alg(onemax, bounds, r_cross, r_mut)
print(f'\nGenetic algorithm completed\n')
print(f'f({best}) : {score}')

Starting genetic algorithm

0> f([1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1]) : -11
0> f([0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1]) : -14
0> f([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0]) : -15
2> f([1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1]) : -16
2> f([1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]) : -17
4> f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1]) : -18
6> f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) : -20

Genetic algorithm completed

f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) : -20
